<a href="https://colab.research.google.com/github/alendoko/bina/blob/main/LB4/DATA/Python_JUPYTER_Data_Analysis_SteyByStep_Melbourne_Houseprice2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jupyter Notebook zur Analyse der Wetterdaten (Sonnenstunden) in der Schweiz der letzten 10 Jahre

## Daten laden, aufbereiten und anreichern

In [1]:
# Importieren der benötigten Bibliotheken
import re
import pandas as pd
import geopandas as gpd
from matplotlib import cm, colors
import plotly.express as px
import plotly.graph_objects as go
import folium
import os

# Dateipfade für die CSV-Dateien
file_path_sun_data = '../data/meteo/meteo_swiss_data.csv'
file_path_stn_data = '../data/meteo/meteo_swiss_stn_data.csv'

# Laden der Daten
sun_data = pd.read_csv(file_path_sun_data, delimiter=';')
stn_data = pd.read_csv(file_path_stn_data, delimiter=';')

# Entfernen von Einträgen für Liechtenstein
stn_data = stn_data[stn_data['canton'] != 'LI']

# Funktion zur Umrechnung von GMS in Dezimalgrad
def gms_to_dg(gms):
    match = re.match(r"(\d+)°(\d+)", gms)
    if match:
        grad = int(match.group(1))
        minuten = int(match.group(2))
        return grad + (minuten / 60)
    raise ValueError("Ungültiges Format. Erwartet: 'XX°YY'")

# Umrechnung der Koordinaten
stn_data['latitude_dg'] = stn_data['latitude'].apply(gms_to_dg)
stn_data['longitude_dg'] = stn_data['longitude'].apply(gms_to_dg)

# Verknüpfen der Wetter- und Stationsdaten
data = pd.merge(sun_data, stn_data, on='stn', how='inner')

# Konvertieren der Zeitspalte in Datumsformat
data['time'] = pd.to_datetime(data['time'], format='%Y%m')
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month

# Ersetzen von '-' durch '0' und Konvertieren in Float
data['sunhours'] = data['su2000m0'].replace('-', '0').astype(float)

# Entfernen ungültiger Werte
data = data[(data['sunhours'] > 0) & (data['year'] != 2025)]

# Gruppieren nach Jahr und Monat
monthly_data = data.groupby(['year', 'month'])['sunhours'].mean().unstack(level=0)

# Monatsnamen definieren
month_names = ['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']

## Diagramm: Wetterstationen in der Schweiz

In [ ]:
# Karte erstellen
map = folium.Map(location=[46.8, 8.33], zoom_start=7, tiles='TopPlusOpen.Color')

# Hinzufügen der Kantonsgrenzen
folium.GeoJson('../data/meteo/swiss_cantons.geojson', name='Kantone').add_to(map)

# Marker für jede Station hinzufügen
for _, stn in stn_data.iterrows():
    folium.Marker(
        location=[stn['latitude_dg'], stn['longitude_dg']],
        popup=stn['stn']
    ).add_to(map)

# Titel hinzufügen
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>Verteilung der Wettermessstationen in der Schweiz</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

# Karte speichern
map.save('../docs/assets/diagramme/swiss_stations_map.html')

# Karte anzeigen
map

## Tabelle, zur Höhenverteilung der Wetterstationen

In [7]:
# Höhenkategorien definieren
bins = [0, 500, 1000, 1500, 2000, 2500, 5000]
labels = ['<500 m', '500–1000 m', '1000–1500 m', '1500–2000 m', '2000–2500 m', '>2500 m']

# Höhenkategorie jeder Station zuweisen
data['elevation_category'] = pd.cut(data['elevation'], bins=bins, labels=labels, right=False)

# Anzahl Stationen pro Höhenkategorie zählen für Jahr 2024
stn_nr_2024 = data[data['year'] == 2024].groupby('elevation_category', observed=True)['stn'].nunique().sort_index()

# Berechnung der jährlichen kumulierten Sonnenstunden pro Höhenkategorie
elevation_monthly_avg = data.groupby(['elevation_category', 'year', 'month'], observed=True)['sunhours'].mean().reset_index()

# Berechnung der jährlichen kumulierten Sonnenstunden pro Kanton
elevation_yearly_cum = elevation_monthly_avg.groupby(['elevation_category', 'year'], observed=True, dropna=False)['sunhours'].sum().reset_index()

# Pivot-Tabelle erstellen, um kumulierte Sonnenstunden pro Jahr anzuzeigen
elevation_yearly_pivot = elevation_yearly_cum.pivot(index='elevation_category', columns='year', values='sunhours')

# Markdown-Tabelle für die kumulierten Werte erstellen
markdown_table_cum = "| Höhenkategorie | Anzahl Stationen<br>(Stand 2024) | " + " | ".join(str(year) for year in elevation_yearly_pivot.columns) + " |\n"
markdown_table_cum += "|:----------------:|:------------------:| " + " | ".join([":------------------:"] * len(elevation_yearly_pivot.columns)) + " |\n"

for category, row in elevation_yearly_pivot.iterrows():
    station_count = stn_nr_2024.get(category, 0)
    # Identify the top 3 values in the row
    top_3_indices = row.nlargest(3).index
    markdown_table_cum += f"| {category} | {station_count:,} | " + " | ".join(
        f"**{value:,.2f}**" if year in top_3_indices else f"{value:,.2f}"
        if not pd.isna(value) else "-" for year, value in row.items()
    ) + " |\n"

# Datei löschen, wenn sie bereits existiert
file_path_cum = "../docs/assets/md/hoehenverteilung_stationen.md"
if os.path.exists(file_path_cum):
    os.remove(file_path_cum)

# In eine .md-Datei schreiben
with open(file_path_cum, "w", encoding="utf-8") as f:
    f.write(markdown_table_cum)

print("Markdown-Datei wurde erfolgreich erstellt: hoehenverteilung_stationen.md")

Markdown-Datei wurde erfolgreich erstellt: hoehenverteilung_stationen.md


## Diagramm erstellen, mit den durchschnittliche Sonnenstunden pro Monat in der Schweiz

In [31]:
# Berechnen des durchschnittlichen Sonnenstunden pro Monat über alle Jahre
average_sunhours = monthly_data.mean(axis=1)

# Berechnen des Medians der Sonnenstunden pro Monat über alle Jahre
median_sunhours = monthly_data.median(axis=1)

# Erstellung eines interaktiven Liniendiagramms mit Plotly
fig = go.Figure()

# Iteriere über die Spalten (Jahre) in den monatlichen Daten und füge eine Linie für jedes Jahr hinzu
for year in monthly_data.columns:
    fig.add_trace(go.Scatter(
        x=monthly_data.index,  # Monatsnamen (Index der Tabelle)
        y=monthly_data[year],  # Sonnenstunden für das jeweilige Jahr
        mode='lines+markers',  # Darstellung als Linie mit Markern
        name=str(year),  # Name des Jahres in der Legende
        opacity=0.5  # Transparenz für bessere Übersichtlichkeit
    ))

# Darstellung des Durchschnitts der Sonnenstunden über alle Jahre
fig.add_trace(go.Scatter(
    x=monthly_data.index,  # Monatsnamen
    y=average_sunhours,  # Durchschnittswerte
    mode='lines',  # Darstellung als Linie
    name='Durchschnitt',  # Name in der Legende
    line=dict(color='black', dash='dash')  # Schwarze gestrichelte Linie
))

# Darstellung des Medians der Sonnenstunden über alle Jahre
fig.add_trace(go.Scatter(
    x=monthly_data.index,  # Monatsnamen
    y=median_sunhours,  # Medianwerte
    mode='lines',  # Darstellung als Linie
    name='Median',  # Name in der Legende
    line=dict(color='red', dash='dot')  # Rote gepunktete Linie
))

# Anpassung des Diagrammlayouts
fig.update_layout(
    title='Durchschnittliche Sonnenstunden pro Monat in der Schweiz',  # Titel des Diagramms
    xaxis_title='Monat',  # Beschriftung der x-Achse
    yaxis_title='Durchschnittliche Sonnenstunden',  # Beschriftung der y-Achse
    xaxis=dict(
        tickmode='array',  # Anpassung der x-Achsen-Ticks
        tickvals=list(range(1, 13)),  # Werte für die Ticks (1 bis 12 für die Monate)
        ticktext=month_names  # Monatsnamen als Beschriftung
    ),
    legend_title='Jahr',  # Titel der Legende
    template='plotly_white'  # Weisses Diagramm-Template
)

# Anzeigen des Diagramms
fig.show()

# Speichern des Diagramms als HTML-Datei
fig.write_html('../docs/assets/diagramme/sunhours_per_month.html')

## Diagramm erstellen, der verteilung der Sonnenstunden pro Monat in der Schweiz

In [ ]:
# Erstellung eines interaktiven Boxplots mit Plotly
# Der Boxplot zeigt die Verteilung der Sonnenstunden pro Monat

# Erstellen des Boxplots
fig_box = px.box(
    data,  # Datensatz mit den Sonnenstunden
    x='month',  # Gruppierung nach Monat
    y='sunhours',  # Werte der Sonnenstunden
    points=False,  # Keine Punkte (Outlier) anzeigen
    labels={'month': 'Monat', 'sunhours': 'Sonnenstunden'}  # Achsenbeschriftungen
)

# Anpassung des Layouts des Boxplots
fig_box.update_layout(
    title='Verteilung der Sonnenstunden pro Monat in der Schweiz',  # Titel des Diagramms
    xaxis=dict(
        tickmode='array',  # Anpassung der x-Achsen-Ticks
        tickvals=list(range(1, 13)),  # Werte für die Ticks (1 bis 12 für die Monate)
        ticktext=month_names  # Monatsnamen als Beschriftung
    ),
    template='plotly_white'  # Weisses Diagramm-Template
)

# Anzeigen des Boxplots
fig_box.show()

# Speichern des Boxplots als HTML-Datei
fig_box.write_html('../docs/assets/diagramme/sunhours_distribution_per_month.html')

## Diagramm erstellen, der durchschnittlichen jährlichen Sonnenstunden pro Kanton

In [ ]:
# Berechnung der monatlichen Durchschnittswerte der Sonnenstunden pro Kanton
canton_monthly_avg = data.groupby(['canton_name', 'year', 'month'])['sunhours'].mean().reset_index()

# Berechnung der jährlichen kumulierten Sonnenstunden pro Kanton
canton_yearly_cum = canton_monthly_avg.groupby(['canton_name', 'year'])['sunhours'].sum().reset_index()

# Berechnung der durchschnittlichen jährlichen Sonnenstunden pro Kanton
canton_avg = canton_yearly_cum.groupby('canton_name')['sunhours'].mean().reset_index()

# Laden der GeoJSON-Daten der Schweizer Kantone
# Diese Daten enthalten die geografischen Grenzen der Kantone
cantons = gpd.read_file('../data/meteo/swiss_cantons.geojson')

# Verknüpfen der Durchschnittswerte mit den GeoJSON-Daten
# Die Verknüpfung erfolgt über den Namen des Kantons
cantons = cantons.merge(canton_avg, left_on='NAME', right_on='canton_name', how='left')

# Erstellung der interaktiven Karte mit Folium
# Festlegen des Startpunkts und der Zoomstufe der Karte
map = folium.Map(location=[46.8, 8.33], zoom_start=7)

# Hinzufügen der Kantonsgrenzen zur Karte
folium.GeoJson(
    '../data/meteo/swiss_cantons.geojson', 
    name='Kantone'
).add_to(map)

# Definition der Farbskala basierend auf den Durchschnittswerten
norm = colors.Normalize(vmin=canton_avg['sunhours'].min(), vmax=canton_avg['sunhours'].max())
colormap = cm.ScalarMappable(norm=norm, cmap='YlOrRd')

# Hinzufügen von Kreis-Markern für die Durchschnittswerte der Sonnenstunden
for _, row in cantons.iterrows():
    if not pd.isna(row['sunhours']):  # Nur wenn ein Durchschnittswert vorhanden ist
        color = colors.to_hex(colormap.to_rgba(row['sunhours']))  # Farbe basierend auf dem Wert
        folium.Circle(
            location=[row['geometry'].centroid.y, row['geometry'].centroid.x],  # Position des Kreises (Kantonszentrum)
            radius=row['sunhours'] * 7,  # Radius des Kreises (Skalierung basierend auf Sonnenstunden)
            color=color,  # Randfarbe des Kreises
            fill=True,  # Kreis ausfüllen
            fill_color=color,  # Füllfarbe des Kreises
            fill_opacity=0.6,  # Transparenz des Kreises
            popup=f"{row['NAME']}: {row['sunhours']:.2f}"  # Popup mit Kanton und Durchschnittswert
        ).add_to(map)

# Hinzufügen von Text-Markern mit den Durchschnittswerten
for _, row in cantons.iterrows():
    if not pd.isna(row['sunhours']):  # Nur wenn ein Durchschnittswert vorhanden ist
        folium.map.Marker(
            [row['geometry'].centroid.y, row['geometry'].centroid.x],  # Position des Textes (Kantonszentrum)
            icon=folium.DivIcon(
                icon_size=(15, 12),  # Grösse des Icons
                icon_anchor=(20, 12),  # Ankerpunkt des Icons
                html=f'<div style="font-size: 12pt; color: black;">{int(row["sunhours"])}</div>'  # Text mit Sonnenstunden
            ),
            popup=f"{row['NAME']}: {row['sunhours']:.2f}"  # Popup mit Kanton und Durchschnittswert
        ).add_to(map)

# HTML-Code für den Titel der Karte
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>Durchschnittliche jährliche Sonnenstunden pro Kanton</b></div>
'''
# Hinzufügen des Titels zur Karte
map.get_root().html.add_child(folium.Element(title_html))

# Speichern der Karte in einer HTML-Datei
# Die Karte wird in einem Ordner gespeichert, um sie später im Browser anzuzeigen
map.save('../docs/assets/diagramme/swiss_sunhours_map.html')

# Anzeigen der Karte (optional, wenn in einer Jupyter-Umgebung verwendet)
map

## Diagramm erstellen, mit den kummulierten Sonnenstunden pro Jahr und Station

In [ ]:
# Gruppieren und kumulieren der Sonnenstunden
data_grouped = data.groupby(["year", "stn", "latitude_dg", "longitude_dg"]).agg({"sunhours": "sum"}).reset_index()
data_grouped["sunhours_cum"] = data_grouped.groupby("stn")["sunhours"].cumsum()

# Liste der Jahre sortieren
years = sorted(data_grouped["year"].unique())

# Farbfunktion
def get_color(value):
    """Bestimmt die Farbe basierend auf dem Wert der kumulierten Sonnenstunden."""
    if value <= 2500: return "rgb(0, 0, 139)"       # Dunkelblau
    elif value <= 5000: return "rgb(30, 144, 255)"  # Blau
    elif value <= 7500: return "rgb(60, 179, 113)"  # Grün
    elif value <= 10000: return "rgb(124, 252, 0)"  # Hellgrün
    elif value <= 12500: return "rgb(255, 255, 0)"  # Gelb
    elif value <= 15000: return "rgb(255, 165, 0)"  # Orange
    elif value <= 17500: return "rgb(255, 69, 0)"   # Rot
    elif value <= 20000: return "rgb(220, 20, 60)"  # Dunkelrot
    else: return "rgb(139, 0, 0)"                   # Sehr dunkelrot

# Linienbreitenfunktion
def get_line_width(value):
    """Bestimmt die Linienbreite basierend auf dem Wert der kumulierten Sonnenstunden."""
    if value < 5000: return 2
    elif value < 10000: return 3
    elif value < 15000: return 4
    elif value < 20000: return 5
    else: return 6

# Start-Frame erstellen
df_start = data_grouped[data_grouped["year"] == years[0]]

# Erstellen der Start-Traces
start_traces = [
    go.Scatter3d(
        x=[row["longitude_dg"]]*2,
        y=[row["latitude_dg"]]*2,
        z=[0, row["sunhours_cum"]],
        mode="lines",
        line=dict(
            color=get_color(row["sunhours_cum"]),
            width=get_line_width(row["sunhours_cum"])
        ),
        text=f"Ort: {row.name}<br>Jahr: {years[0]}<br>Sonnenstunden: {int(row.sunhours_cum):,}".replace(',', ''),
        hoverinfo="text",
        showlegend=False
    ) for _, row in df_start.iterrows()
]

# Laden der GeoJSON-Daten für die Schweizerkarte
swiss_map = gpd.read_file('../data/meteo/swiss_cantons.geojson')

# Extrahieren der Koordinaten für die Kantonsgrenzen
map_lines = []
for _, row in swiss_map.iterrows():
    if row['geometry'].geom_type == 'Polygon':
        x, y = row['geometry'].exterior.xy
        map_lines.append(go.Scatter3d(
            x=x.tolist(),
            y=y.tolist(),
            z=[0] * len(x),
            mode='lines',
            line=dict(color='black', width=2),
            name=row['NAME'],
            showlegend=False
        ))
    elif row['geometry'].geom_type == 'MultiPolygon':
        for polygon in row['geometry']:
            x, y = polygon.exterior.xy
            map_lines.append(go.Scatter3d(
                x=x,
                y=y,
                z=[0] * len(x),
                mode='lines',
                line=dict(color='black', width=2),
                name=row['NAME'],
                showlegend=False
            ))

# Hinzufügen der Schweizerkarte als Basis zu den Start-Traces
start_traces.extend(map_lines)

# Animations-Frames erstellen
frames = [
    go.Frame(
        data=[
            go.Scatter3d(
                x=[row["longitude_dg"]]*2,
                y=[row["latitude_dg"]]*2,
                z=[0, row["sunhours_cum"]],
                mode="lines",
                line=dict(
                    color=get_color(row["sunhours_cum"]),
                    width=get_line_width(row["sunhours_cum"])
                ),
                text=f"Ort: {row.name}<br>Jahr: {jahr}<br>Sonnenstunden: {int(row.sunhours_cum):,}",
                hoverinfo="text",
                showlegend=False
            ) for _, row in data_grouped[data_grouped["year"] == jahr].head(110).iterrows()
        ] + map_lines,  # Add map_lines to every frame
        name=str(jahr)
    ) for jahr in years
]

# Layout erstellen
layout = go.Layout(
    title="☀️ Kumulierte Sonnenstunden in der Schweiz der letzten 10 Jahre",
    scene=dict(
        xaxis=dict(
            title="Längengrad", 
            showgrid=True, 
            backgroundcolor="rgba(0, 0, 0,0)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white"),
        yaxis=dict(
            title="Breitengrad", 
            backgroundcolor="rgba(0, 0, 0,0)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white"),
        zaxis=dict(
            title="Kumulierte Sonnenstunden",
            tickvals=[5000, 10000, 15000, 20000],
            ticktext=["5'000", "10'000", "15'000", "20'000"],
            range=[0, data_grouped["sunhours_cum"].max()],
            backgroundcolor="rgba(0, 0, 0,0)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white"
        ),
        camera=dict(
            eye=dict(x=0.5, y=-2.5, z=1.5)
        )
    ),
    sliders=[{
        "steps": [{
            "method": "animate",
            "label": str(j),
            "args": [[str(j)], {"mode": "immediate", "frame": {"duration": 600, "redraw": True}}]
        } for j in years]
    }],
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {"label": "Play", "method": "animate", "args": [None]},
            {"label": "Pause", "method": "animate", "args": [[None], {"mode": "immediate", "frame": {"duration": 0}}]}
        ]
    }]
)

# Diagramm erstellen und speichern 
fig = go.Figure(data=start_traces, layout=layout, frames=frames)
fig.write_html("../docs/assets/diagramme/sonnenstunden_3d.html")
fig.show()
